In [2]:
import re
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
import pickle


In [3]:
def extract_numbers_from_line(line):
    # Use regular expressions to find all numeric values in the line
    for i in range(1,6):
        string_replace = f"'{i}':"
        line=line.replace(string_replace,'')
    #print('Line, replaced', line)
    numbers = re.findall(r'-?\d+', line)
    #print('numbers', numbers)
    output=[]
    for num in numbers:
        output.append(int(num))
    return output


In [12]:
A= X[:,1].reshape(-1,1)
B= y[:,1].reshape(-1,1)

A_0= X[:,0].reshape(-1,1)
B_0= y[:,0].reshape(-1,1)
indexes_to_use=[]
for i in range(A.shape[0]):
    print(f'Line {i}, dict:{A[i]}, change{B[i]}')
    if abs(A_0[i])<10:
        indexes_to_use.append(i)
print(indexes_to_use_joints1)
#14, 15,16,19,57,1,21,22,30,32,33,34,35,62,57,

Line 0, dict:[-48], change[-2402]
Line 1, dict:[0], change[27]
Line 2, dict:[0], change[-3]
Line 3, dict:[0], change[-3]
Line 4, dict:[66], change[3285]
Line 5, dict:[-13], change[-623]
Line 6, dict:[0], change[-10]
Line 7, dict:[0], change[2]
Line 8, dict:[-63], change[-2909]
Line 9, dict:[74], change[3447]
Line 10, dict:[0], change[-16]
Line 11, dict:[0], change[0]
Line 12, dict:[0], change[1]
Line 13, dict:[0], change[0]
Line 14, dict:[90], change[260]
Line 15, dict:[-48], change[-519]
Line 16, dict:[-106], change[-261]
Line 17, dict:[67], change[1065]
Line 18, dict:[-25], change[-1363]
Line 19, dict:[-82], change[-759]
Line 20, dict:[23], change[1358]
Line 21, dict:[128], change[1525]
Line 22, dict:[116], change[1569]
Line 23, dict:[1], change[1]
Line 24, dict:[57], change[1567]
Line 25, dict:[64], change[0]
Line 26, dict:[0], change[1]
Line 27, dict:[0], change[0]
Line 28, dict:[0], change[4]
Line 29, dict:[-56], change[-2660]
Line 30, dict:[0], change[45]
Line 31, dict:[-62], cha

In [28]:
x_dataset=[]
y_dataset=[]
with open('Data_aquisition_29_12_Joints copy.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        numbers = extract_numbers_from_line(line)
        dic_values=[0,0,0,0,0]
        deltas= numbers[10:]
        dictionary= numbers[:10]
        print(deltas , dictionary)
        for i in range(5):
            #print('Number',number)
                if i==0:
                    dic_values[i] = dictionary[0]-dictionary[1]
                elif i==1:
                    dic_values[i] = dictionary[2]-dictionary[3]
                elif i==2:
                    dic_values[i]  = dictionary[4]-dictionary[5]
                elif i==3:
                    dic_values[i]  = dictionary[6]-dictionary[7]
                elif i==4:
                    dic_values[i]  = dictionary[8]-dictionary[9]
        x_dataset.append(dic_values)
        y_dataset.append(deltas)

X= np.array(x_dataset)
y= np.array(y_dataset)

   

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test.shape
models={}
for i in range(X_train.shape[1]):
    if i ==1:
        X_1=X[indexes_to_use,i].reshape(-1,1)
        y_1=y[indexes_to_use,i].reshape(-1,1)
        x_train_column, x_test_column, y_train_column, y_test_column = train_test_split(X_1, y_1, test_size=0.2, random_state=42)
    else:
        x_train_column = X_train[:,i].reshape(-1,1)
        y_train_column = y_train[:,i].reshape(-1,1)
        x_test_column = X_test[:,i].reshape(-1,1)
        y_test_column = y_test[:,i].reshape(-1,1)
        
    modelRidge = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1], fit_intercept=False)
    modelRidge.fit(x_train_column,y_train_column)
    y_pred_test_R = modelRidge.predict(x_test_column)
    mseR = mean_squared_error(y_test_column, y_pred_test_R)

    modelSimple = LinearRegression(fit_intercept=False)
    modelSimple.fit(x_train_column,y_train_column)
    y_pred_test_S = modelSimple.predict(x_test_column)
    mseS = mean_squared_error(y_test_column, y_pred_test_S)        

    if mseR>mseS:
        model=modelSimple
        mse=mseS
        y_pred_test=y_pred_test_S
        print('\n\nLin Reg')
    else:
        model=modelRidge
        mse=mseR
        y_pred_test=y_pred_test_R
        print('\n\nRidge')

    print(f'\nJoint {i}, Mean Squared Error: {mse}\n')
    for k in range(x_test_column.shape[0]):
        print(f'dictionary:{x_test_column[k]}, true:{y_test_column[k]}  predicted:{y_pred_test[k]}\n')
    if i==0:
        models['Joints_model0']=model
    elif i==1:
        models['Joints_model1']=model
    elif i==2:
        models['Joints_model2']=model
    elif i==3:
        models['Joints_model3']=model
    elif i==4:
        models['Joints_model4']=model
    
with open("Joints_models.pickle", "wb") as f:
    pickle.dump(models, f)
    





[0, -2402, -3, 0, 0] [0, 0, 0, 48, 0, 0, 0, 0, 0, 0]
[0, 27, 1, -4122, 0] [0, 0, 0, 0, 0, 0, 38, 0, 0, 0]
[0, -3, 1, 7061, 0] [0, 0, 0, 0, 0, 0, 0, 68, 1, 0]
[0, -3, 0, 9, -2655] [0, 0, 0, 0, 0, 0, 0, 0, 0, 51]
[0, 3285, 0, 0, 2] [0, 0, 66, 0, 0, 0, 0, 0, 0, 0]
[1490, -623, 0, 0, 0] [23, 0, 1, 14, 0, 0, 0, 0, 0, 0]
[1478, -10, 0, 0, 0] [21, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-2391, 2, 0, 0, 0] [1, 53, 0, 0, 0, 0, 0, 0, 0, 0]
[-282, -2909, 0, 0, -1] [0, 1, 0, 63, 0, 0, 0, 0, 0, 0]
[0, 3447, 0, 0, 1] [0, 0, 74, 0, 0, 0, 0, 0, 0, 0]
[0, -16, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-1, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, -4699, -1] [0, 0, 0, 0, 0, 0, 50, 0, 0, 0]
[1, 0, 0, 6779, 0] [0, 0, 0, 0, 0, 0, 0, 74, 1, 0]
[-1295, 260, 0, 11, 1] [28, 58, 90, 0, 0, 0, 0, 0, 0, 0]
[1564, -519, 0, -1, 0] [41, 0, 0, 48, 0, 0, 0, 0, 0, 0]
[2363, -261, 0, 1, 0] [78, 23, 0, 106, 0, 0, 0, 0, 0, 0]
[2081, 1065, 0, 1, 0] [47, 7, 68, 1, 0, 0, 0, 0, 0, 0]
[-9, -1363, 0, 0, -1] [0, 0, 0, 25, 0, 0, 0, 

In [14]:
models

{'Joints_model0': LinearRegression(fit_intercept=False),
 'Joints_model1': LinearRegression(fit_intercept=False),
 'Joints_model2': LinearRegression(fit_intercept=False),
 'Joints_model3': RidgeCV(alphas=[0.001, 0.01, 0.1, 1], fit_intercept=False),
 'Joints_model4': RidgeCV(alphas=[0.001, 0.01, 0.1, 1], fit_intercept=False)}

In [16]:
with open("Joints_models.pickle", "rb") as f:
    while True:
        try:
            models_loaded=pickle.load(f)
            print(models_loaded)
        except EOFError:
            break

models_loaded

{'Joints_model0': LinearRegression(fit_intercept=False), 'Joints_model1': LinearRegression(fit_intercept=False), 'Joints_model2': LinearRegression(fit_intercept=False), 'Joints_model3': RidgeCV(alphas=[0.001, 0.01, 0.1, 1], fit_intercept=False), 'Joints_model4': RidgeCV(alphas=[0.001, 0.01, 0.1, 1], fit_intercept=False)}


{'Joints_model0': LinearRegression(fit_intercept=False),
 'Joints_model1': LinearRegression(fit_intercept=False),
 'Joints_model2': LinearRegression(fit_intercept=False),
 'Joints_model3': RidgeCV(alphas=[0.001, 0.01, 0.1, 1], fit_intercept=False),
 'Joints_model4': RidgeCV(alphas=[0.001, 0.01, 0.1, 1], fit_intercept=False)}

In [24]:
joints=[0,0,0,0,0]
deltas=[0,23,-4,0,10]
for i in range(len(joints)):
    model= models_loaded[f'Joints_model{i}']
    delta = np.array([deltas[i]]).reshape(-1,1)
    change_joints=int(model.predict(delta))
    joints[i]+= change_joints
print(joints)

[0, 1057, 173, 0, 300]


C:\Users\alexa\AppData\Local\Temp\ipykernel_23572\2154408099.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  change_joints=int(model.predict(delta))
C:\Users\alexa\AppData\Local\Temp\ipykernel_23572\2154408099.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  change_joints=int(model.predict(delta))
C:\Users\alexa\AppData\Local\Temp\ipykernel_23572\2154408099.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  change_joints=int(model.predict(delta))
C:\Users\a

In [39]:
a=np.array([21]).reshape(-1,1)
print(a.shape)
b= models_loaded['Joints_model0'].predict(a)
print(b)

(1, 1)
[[922.96280662]]


In [4]:
x_dataset=[]
y_dataset=[]
with open('Data_aquisition_29_12_xyz.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        numbers = extract_numbers_from_line(line)
        dic_values=[0,0,0,0,0]
        deltas= numbers[10:]
        dictionary= numbers[:10]
        print(deltas , dictionary)
        for i in range(5):
            #print('Number',number)
                if i==0:
                    dic_values[i] = dictionary[0]-dictionary[1]
                elif i==1:
                    dic_values[i] = dictionary[2]-dictionary[3]
                elif i==2:
                    dic_values[i]  = dictionary[4]-dictionary[5]
                elif i==3:
                    dic_values[i]  = dictionary[6]-dictionary[7]
                elif i==4:
                    dic_values[i]  = dictionary[8]-dictionary[9]
        x_dataset.append(dic_values)
        y_dataset.append(deltas)

X= np.array(x_dataset)
y= np.array(y_dataset)

   

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test.shape
models={}
for i in range(X_train.shape[1]):
    
    x_train_column = X_train[:,i].reshape(-1,1)
    y_train_column = y_train[:,i].reshape(-1,1)
    x_test_column = X_test[:,i].reshape(-1,1)
    y_test_column = y_test[:,i].reshape(-1,1)
        
    modelRidge = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1], fit_intercept=False)
    modelRidge.fit(x_train_column,y_train_column)
    y_pred_test_R = modelRidge.predict(x_test_column)
    mseR = mean_squared_error(y_test_column, y_pred_test_R)

    modelSimple = LinearRegression(fit_intercept=False)
    modelSimple.fit(x_train_column,y_train_column)
    y_pred_test_S = modelSimple.predict(x_test_column)
    mseS = mean_squared_error(y_test_column, y_pred_test_S)        

    if mseR>mseS:
        model=modelSimple
        mse=mseS
        y_pred_test=y_pred_test_S
        print('\n\nLin Reg')
    else:
        model=modelRidge
        mse=mseR
        y_pred_test=y_pred_test_R
        print('\n\nRidge')

    print(f'\Axis {i}, Mean Squared Error: {mse}\n')
    for k in range(x_test_column.shape[0]):
        print(f'dictionary:{x_test_column[k]}, true:{y_test_column[k]}  predicted:{y_pred_test[k]}\n')
    if i==0:
        models['XYZ_0']=model
    elif i==1:
        models['XYZ_1']=model
    elif i==2:
        models['XYZ_2']=model
    elif i==3:
        models['XYZ_3']=model
    elif i==4:
        models['XYZ_4']=model
    
with open("XYZ_models.pickle", "wb") as f:
    pickle.dump(models, f)
    





[-726, 2, 380, -1, 0] [164, 0, 0, 0, 48, 147, 0, 0, 0, 0]
[2, -1, -884, 1, 0] [18, 16, 0, 0, 207, 0, 0, 0, 0, 0]
[-4, -915, -171, -1, -39] [0, 0, 212, 0, 32, 0, 0, 0, 0, 11]
[2, 1251, 5, 0, 0] [0, 0, 0, 290, 0, 0, 0, 0, 0, 0]
[-361, -5, 3, 1, 0] [80, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1555, 4, -8, -1, 0] [0, 355, 0, 2, 0, 0, 0, 0, 0, 0]
[279, -197, 6, 1, 0] [9, 81, 71, 0, 0, 0, 0, 0, 0, 0]
[0, -559, -5, 0, 0] [0, 0, 130, 0, 0, 0, 0, 0, 0, 0]
[-5, 6, -1030, -2, 0] [0, 0, 0, 0, 258, 11, 0, 0, 0, 0]
[3, 1169, 8, 1, 0] [15, 14, 24, 292, 0, 0, 0, 0, 0, 0]
[227, -197, -4, 0, 0] [0, 53, 47, 0, 0, 0, 0, 0, 0, 0]
[-1177, -2, 3, 0, 0] [274, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-408, -32, 6, 1, 0] [101, 0, 2, 0, 0, 0, 0, 0, 0, 0]
[417, -807, -5, 0, 0] [0, 90, 255, 1, 0, 0, 0, 0, 0, 0]
[-110, 59, 255, 0, 0] [30, 0, 0, 33, 0, 57, 0, 0, 0, 0]
[-250, 27, 249, 0, 0] [68, 0, 0, 68, 0, 71, 0, 0, 0, 0]
[556, -83, -5, 0, 0] [0, 127, 101, 0, 0, 0, 0, 0, 19, 0]
[-10, 6, -3, 191, -20] [0, 0, 1, 0, 0, 0, 25, 434, 4, 67]
[2, 

In [21]:
xJ_dataset=[]
yJ_dataset=[]
with open('Data_aquisition_29_12_xyz.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        numbers = extract_numbers_from_line(line)
        dic_values=[0,0,0,0,0]
        deltas= numbers[10:]
        dictionary= numbers[:10]
        #print(deltas , dictionary)
        for i in range(5):
            #print('Number',number)
                if i==0:
                    dic_values[i] = dictionary[0]-dictionary[1]
                elif i==1:
                    dic_values[i] = dictionary[2]-dictionary[3]
                elif i==2:
                    dic_values[i]  = dictionary[4]-dictionary[5]
                elif i==3:
                    dic_values[i]  = dictionary[6]-dictionary[7]
                elif i==4:
                    dic_values[i]  = dictionary[8]-dictionary[9]
        x_dataset.append(dic_values)
        y_dataset.append(deltas)


X= np.array(x_dataset)
y= np.array(y_dataset)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test.shape
# Create a linear regression model
model = LinearRegression()

# Wrap the linear regression model with MultiOutputRegressor
multioutput_model = MultiOutputRegressor(model)

multioutput_model.fit(X_train, y_train)

y_pred_test = multioutput_model.predict(X_test)


# Evaluate the model using mean squared error (you can use other metrics as needed)
mse = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error: {mse}')

for i in range(X_test.shape[0]):
    print(f'dictionary:{X_test[i,:]}, deltaTrue:{y_test[i,:]}, deltapredicted:{y_pred_test[i,:]}\n')

Mean Squared Error: 2877167.7439640807
dictionary:[108 -67  73   0   0], deltaTrue:[ 4675     0 -3134     0     0], deltapredicted:[ 1931.24566615  -392.91908602 -1335.41625229   -35.95028965
  -107.57313928]

dictionary:[  80 -117    0    0    0], deltaTrue:[ 3646 -1611     0    -5     0], deltapredicted:[ 903.37592914 -605.87547603   21.48063911 -164.2658351   -78.01153796]

dictionary:[  15 -174    0    0  -23], deltaTrue:[-86 727   0  -1 -20], deltapredicted:[-593.59601106 -857.28287848  135.28128528  361.8063074  -199.31984354]

dictionary:[  0 -25   0   0   0], deltaTrue:[   -9 -1363     0     0    -1], deltapredicted:[-158.89466699 -182.63499042   83.80833184  -41.68746416  -57.52474089]

dictionary:[   1 -268    0    0    0], deltaTrue:[   3 1169    8    1    0], deltapredicted:[-1272.35099654 -1294.98567847   239.55957699   -82.3067922
   -70.54597903]

dictionary:[274   0   0   0   0], deltaTrue:[-1177    -2     3     0     0], deltapredicted:[5063.8455341   -75.43742499 -349